# Examples with the Sunpy package

## Goals

- Get used to it.
- Prepare the terrain for the sunspot to solar system locator routine.
- Access solar events

In [78]:
from sunpy.net import attrs as a
from sunpy.net import Fido
tstart = '2022/08/30 16:00:00'
tend = '2022/08/31 03:00:00'
event_type = 'ER'
result = Fido.search(a.Time(tstart,tend), a.hek.EventType(event_type))
print(len(result), result)

1 Results from 1 Provider:

4 Results from the HEKClient:

                                                            gs_thumburl                                                             ...
----------------------------------------------------------------------------------------------------------------------------------- ...
 http://sdowww.lmsal.com/sdomedia/ssw/media/ssw/ssw_client/data/ssw_service_220830_104147_1898/www/ssw_cutout_304_context_thumb.gif ...
http://sdowww.lmsal.com/sdomedia/ssw/media/ssw/ssw_client/data/ssw_service_230225_071103_41466/www/ssw_cutout_304_context_thumb.gif ...
http://sdowww.lmsal.com/sdomedia/ssw/media/ssw/ssw_client/data/ssw_service_220830_112146_29253/www/ssw_cutout_304_context_thumb.gif ...
http://sdowww.lmsal.com/sdomedia/ssw/media/ssw/ssw_client/data/ssw_service_220830_152006_35994/www/ssw_cutout_304_context_thumb.gif ...




In [79]:
print(result["hek"]['event_coord1'])
print(result["hek"]['event_coord2'])
print(result["hek"]['event_coordsys'])
print(result["hek"]['obs_observatory'])



   event_coord1   
------------------
             762.0
 855.1129760742188
             786.0
-946.7999877929688
   event_coord2   
------------------
-572.4000244140625
-482.6319885253906
-548.4000244140625
 339.6000061035156
event_coordsys
--------------
  UTC-HPC-TOPO
  UTC-HPC-TOPO
  UTC-HPC-TOPO
  UTC-HPC-TOPO
obs_observatory
---------------
            SDO
            SDO
            SDO
            SDO


In [119]:
import matplotlib.pyplot as plt
import numpy as np
from skyfield.api import load
from skyfield import units
from astropy.time import Time
from astropy import units as u
from datetime import timedelta
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
# Create a timescale and initialise beginning and end time.
ts = load.timescale()
t = ts.utc(2012, 3, 6)

# Load the JPL ephemeris DE421 (covers 1900-2050).
eph = load('../de421.bsp')
sun, earth, venus = eph['sun'], eph['earth'], eph['venus']

# Compute the position of Venus in the ICRS frame.
venus_positions = []
venus_times = []
venus_pos = venus.at(t)
print(venus_pos)
ra, dec, distance = venus_pos.radec()
print(venus_pos.radec())
print(ra, dec)
dec = dec.to(u.deg)
ra = ra.to(u.deg)
distance = distance.to(u.au)
venus_times.append(t)
c = SkyCoord(ra, dec, frame='icrs')
print(c)

# Compute the position of Venus in the Heliographic stonyhurst frame.
#print(c.transform_to(frames.Helioprojective(observer='earth'))) #this is cartesian. We need angles to be sure that this corresponds to the sunspot.

<Barycentric BCRS position and velocity at date t center=0 target=299>
(<Angle 07h 16m 13.66s>, <Angle +24deg 01' 16.1">, <Distance 0.718214 au>)
07h 16m 13.66s +24deg 01' 16.1"
<SkyCoord (ICRS): (ra, dec) in deg
    (109.05692532, 24.02114302)>


In [115]:
print(c)

<SkyCoord (ICRS): (ra, dec) in deg
    (316.6326902, -19.54388683)>


In [121]:
import astroquery.heasarc
from astroquery.heasarc import Heasarc, Conf
from astropy.io import fits
from scipy import stats
import astropy.table

Heasarc = astroquery.heasarc.Heasarc()
Conf.server.set('https://www.isdc.unige.ch/browse/w3query.pl')
table = Heasarc.query_mission_list()
table.pprint(max_width=120)

def get_scw_list(coord,radius,start_date,end_date ):
    R = Heasarc.query_region(
            coord,
            mission = 'integral_rev3_scw',
            radius=radius,
            time = start_date + ' .. ' + end_date,
            good_isgri = ">1000",
        )
    
assert astroquery.__version__ >= '0.4.2.dev6611'

# it means it's our fork
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

radius = '15 degree'
time = '2012-03-05T00:00:00'
delta_time = '2012-03-08T00:00:00'
scw_list = get_scw_list(c,radius,time,delta_time)
print(scw_list)

   Mission            Table                         Table Description               
------------- ---------------------- -----------------------------------------------
CTASST1M-REV1     cta_sst1m_rev1_run                                             Run
    FACT-REV1          fact_rev1_run                                             Run
INTEGRAL-REV3     integral_rev3_prop                                       Proposals
INTEGRAL-REV3 integral_rev3_prop_obs Proposal Information and Observation Parameters
INTEGRAL-REV3      integral_rev3_scw                       SCW - Science Window Data


None
